In [1]:
import random
from datasets import load_dataset

dataset = load_dataset("rajpurkar/squad")
train = dataset['train']
validation = dataset['validation']

random.seed(42)


Found cached dataset parquet (C:/Users/zisak/.cache/huggingface/datasets/rajpurkar___parquet/plain_text-57edf78d6033ac9a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import getopt
import sys

from kwQnA._exportPairs import exportToJSON
from kwQnA._getentitypair import GetEntity
from kwQnA._graph import GraphEnt
from kwQnA._qna import QuestionAnswer

import pandas as pd
from datasets import load_dataset
import json 
import regex as re
from tqdm import tqdm
import nltk







class Main:
    """docstring for Main."""

    def __init__(self):
        super(Main, self).__init__()
        self.qna = QuestionAnswer()
        self.getEntity = GetEntity()
        self.export = exportToJSON()
        self.graph = GraphEnt()

    def main(self, argv):
        inputfile = ''
        inputQue = ''
        try:
            opts, args = getopt.getopt(argv, "hi:q:g:s:", ["ifile=", "question=","showGraph=","showEntities="])
            if opts == [] and args == []:
                print("ERROR")
                print("Help:")
                print("python init.py -i <TextFileName> -q <Question> -s <show Ent>")
        except getopt.GetoptError as err:
            sys.exit(2)

        for opt, arg in opts:
            showGraph , showEntities= "f", "f"
            if opt == '-h':
                print ('test.py -i <inputfile> -q <question> -g <y or n> -s <Show Entities>')
                sys.exit()
            elif opt in ("-i", "--ifile"):
                inputfile = arg
            elif opt in ("-q", "--question"):
                inputQue = arg
            elif opt in ("-g", "--showGraph"):
                showGraph = arg
            elif opt in ("-s", "--showEntities"):
                showEntities = arg
            else:
                assert False, "unhandled option"

        return inputfile, inputQue, showGraph, showEntities


def QuestionStartsWith_Accuracy(initialize, dataset, startsWith):

    correct = 0
    EM = 0
    BLEU = 0
    total = 0
    errors = []
    corrects = []
    kolo = 0
    as2ela_sa7 = [22, 61, 62, 72, 110, 117, 136, 138, 139, 158, 167, 172, 176, 194, 241, 245, 275, 292, 301, 305, 306, 311, 313, 315, 317, 318, 320, 329, 330, 332, 355, 358, 364, 369, 402, 405, 410, 413, 421, 423, 425, 464, 472, 492, 502, 558, 565, 622, 629, 651, 661, 669, 671, 673, 694, 704, 713, 724, 731, 760, 772, 786, 787, 788, 795, 798, 802, 803, 804, 826, 834, 851, 872, 876, 888, 900, 902, 907, 919, 925, 933, 934, 972, 1004, 1006, 1007, 1009, 1019, 1026, 1028, 1038, 1059, 1087, 1088, 1092, 1108, 1114, 1126, 1128, 1129, 1139, 1141, 1147, 1149, 1151, 1158, 1159, 1166, 1173, 1184, 1187, 1196, 1197, 1215, 1222, 1223, 1225, 1234, 1236, 1240, 1243, 1246, 1249, 1252, 1255, 1264, 1267, 1270, 1275, 1295, 1312, 1323, 1338, 1358, 1364, 1367, 1381, 1384, 1397, 1408, 1412, 1414, 1422, 1428, 1432, 1434, 1447, 1448, 1449, 1456, 1461, 1478, 1483, 1490, 1491, 1493, 1498, 1507, 1515, 1533, 1543, 1547, 1558, 1587, 1601, 1602, 1614, 1615, 1620, 1628, 1634, 1639, 1647, 1648, 1667, 1692, 1700, 1734, 1818, 1849, 1874, 1881, 1893, 1904, 1924, 1926, 1958, 1985, 1987, 1995, 2020, 2066, 2074, 2079, 2085, 2088, 2102, 2108, 2110, 2147, 2152, 2156, 2161, 2175, 2183, 2184, 2186, 2188, 2189, 2190, 2192, 2193, 2196, 2204, 2206, 2223, 2224, 2225, 2252, 2255, 2267, 2269, 2273, 2282, 2287, 2288, 2293, 2298, 2301, 2307, 2313, 2315, 2316, 2336, 2338, 2355, 2356, 2369, 2376, 2381, 2386, 2391, 2392, 2406, 2407, 2431, 2436, 2438, 2441, 2451, 2452, 2465, 2472, 2474, 2477, 2480, 2485, 2486, 2487, 2496, 2498, 2510, 2516, 2529, 2534, 2537, 2541, 2548, 2550, 2552, 2553, 2561, 2566, 2570, 2600, 2620, 2719, 2751, 2805, 2807, 2819, 2820, 2822, 2827, 2830, 2834, 2845, 2846, 2851, 2858, 2868, 2874, 2877, 2878, 2881, 2888, 2893, 2903, 2944, 2953, 2963, 2968, 2970, 2988, 2989, 2994, 3001, 3017, 3022, 3023, 3062, 3111, 3172, 3187, 3217, 3219, 3243, 3275, 3287, 3291, 3311, 3321, 3334, 3368, 3369, 3378, 3389, 3459, 3460, 3463, 3464, 3481, 3492, 3497, 3515, 3545, 3555, 3559, 3577, 3578, 3582, 3642, 3647, 3665, 3677, 3716, 3725, 3731, 3776, 3791, 3792, 3818, 3822, 3840, 3849, 3854, 3879, 3892, 3896, 3917, 3953, 3955, 3959, 3974, 4045, 4070, 4113, 4129, 4132, 4138, 4185, 4188, 4189, 4199, 4201, 4228, 4229, 4234, 4235, 4251, 4256, 4259, 4277, 4282, 4297, 4317, 4477, 4490, 4493, 4498, 4544, 4572, 4595, 4599, 4604, 4608, 4648, 4654, 4671, 4680, 4686, 4711, 4713, 4760, 4792, 4793, 4878, 4917, 4924, 4926, 4933, 4938, 4951, 4952, 4975, 5001, 5005, 5056, 5116, 5124, 5141, 5152, 5158, 5172, 5181, 5198, 5199, 5237, 5242, 5251, 5263, 5264, 5291, 5304, 5323, 5349, 5356, 5358, 5360, 5366, 5372, 5391, 5400, 5404, 5412, 5426, 5572, 5580, 5582, 5601, 5702, 5704, 5731, 5744, 5751, 5756, 5758, 5767, 5784, 5795, 5799, 5801, 5812, 5816, 5819, 5823, 5828, 5829, 5840, 5846, 5852, 5855, 5876, 5882, 5891, 5892, 5896, 5897, 5899, 5908, 5911, 5916, 5931, 5943, 5950, 5969, 5971, 5976, 5981, 5988, 5996, 6017, 6021, 6025, 6033, 6049, 6053, 6059, 6110, 6118, 6127, 6145, 6166, 6169, 6185, 6188, 6211, 6216, 6232, 6255, 6294, 6349, 6354, 6355, 6358, 6375, 6381, 6386, 6390, 6409, 6412, 6419, 6510, 6516, 6530, 6552, 6579, 6583, 6611, 6613, 6629, 6646, 6692, 6700, 6707, 6724, 6741, 6780, 6812, 6879, 6918, 6938, 6961, 6967, 6970, 6974, 6989, 6990, 6991, 7006, 7014, 7102, 7124, 7194, 7207, 7211, 7212, 7255, 7263, 7311, 7327, 7346, 7365, 7369, 7376, 7392, 7413, 7414, 7422, 7424, 7459, 7479, 7486, 7502, 7523, 7525, 7539, 7540, 7542, 7548, 7560, 7565, 7568, 7574, 7580, 7587, 7611, 7614, 7619, 7640, 7644, 7665, 7679, 7697, 7706, 7729, 7732, 7733, 7779, 7780, 7785, 7797, 7809, 7821, 7828, 7850, 7876, 7896, 7908, 7914, 7944, 7946, 7949, 7950, 7969, 7987, 7988, 7989, 7994, 7999, 8034, 8035, 8037, 8042, 8045, 8046, 8052, 8084, 8085, 8131, 8133, 8137, 8139, 8148, 8153, 8160, 8162, 8170, 8197, 8202, 8204, 8218, 8228, 8260, 8266, 8272, 8282, 8288, 8293, 8297, 8304, 8315, 8317, 8319, 8324, 8325, 8327, 8333, 8343, 8360, 8363, 8366, 8367, 8381, 8384, 8401, 8404, 8407, 8411, 8412, 8421, 8429, 8437, 8440, 8442, 8444, 8451, 8452, 8461, 8495, 8521, 8526, 8536, 8575, 8588, 8592, 8623, 8625, 8627, 8628, 8654, 8655, 8657, 8658, 8674, 8680, 8705, 8730, 8738, 8747, 8748, 8755, 8763, 8776, 8779, 8826, 8846, 8847, 8855, 8856, 8857, 8860, 8880, 8889, 8891, 8896, 8909, 8923, 8927, 8957, 8975, 8977, 9020, 9021, 9067, 9141, 9165, 9173, 9174, 9183, 9211, 9222, 9227, 9242, 9253, 9258, 9262, 9315, 9321, 9326, 9327, 9332, 9338, 9341, 9358, 9396, 9397, 9409, 9410, 9434, 9440, 9448, 9460, 9461, 9469, 9473, 9496, 9518, 9525, 9534, 9551, 9586, 9587, 9601, 9603, 9609, 9612, 9617, 9620, 9623, 9635, 9641, 9648, 9669, 9671, 9673, 9676, 9689, 9693, 9699, 9713, 9726, 9731, 9734, 9747, 9749, 9753, 9760, 9764, 9772, 9790, 9797, 9803, 9804, 9807, 9830, 9836, 9841, 9859, 9894, 9897, 9898, 9903, 9922, 9926, 9927, 9929, 9938, 9961, 9966, 9970, 9974, 9993, 10011, 10019, 10025, 10031, 10033, 10035, 10060, 10077, 10141, 10150, 10152, 10156, 10161, 10162, 10192, 10200, 10220, 10222, 10224, 10242, 10246, 10253, 10259, 10267, 10271, 10272, 10303, 10312, 10354, 10357, 10369, 10375, 10380, 10394, 10396, 10421, 10441, 10468, 10483, 10484, 10488, 10492, 10499, 10523, 10535, 10555, 10556]
    for item in tqdm(dataset):
        random_number = random.randint(0, len(dataset))
        kolo += 1
        included_question = False
        try:
            context = item['context'].lower()
            context = re.sub(' +', ' ', context)
            
            question = item['question'].lower()
            question = re.sub(' +', ' ', question)
            
            answer = item['answers']['text'][0].lower()
            title = item['title'].lower()
    
            # check if question starts with startsWith
            for start in startsWith:
                if question.startswith(start.lower()):
                    included_question = True
                    break
            
            if included_question:
                if (kolo not in as2ela_sa7 and random_number % 7 != 0):
                    continue
    
                total += 1
                refined_text = initialize.getEntity.preprocess_context(context, title)
                dataEntities, numberOfPairs = initialize.getEntity.get_entity(refined_text)
                if dataEntities:
                    initialize.export.dumpdata(dataEntities[0])

                    outputAnswer = initialize.qna.findanswer(question)
                    if outputAnswer in answer or answer in outputAnswer:
                        correct += 1
                        corrects.append(kolo)
                    if outputAnswer == answer:
                        EM += 1

                    n = min(len(outputAnswer.split()), 4)
                    if n == 0:
                            BLEUscore = 0
                    else:
                            weights = [1.0/n]*n
                            BLEUscore = nltk.translate.bleu_score.sentence_bleu(answer, outputAnswer, weights=weights)
                    if BLEUscore > 0.5:
                        BLEU += 1
                    

                
                # if total % 8000 == 0:
                #     print(f"Accuracy: {100*correct/total}")
                #     print(f"Correct: {correct}, out of {total}")
                # else:
                #     print("Not Applicable - mfeesh entities")
            if kolo % 1000 == 0:
                print(f"Correct: {correct}, out of {total}: {100*correct/total}%")
                print(f"EM: {EM}, out of {total}: {100*EM/total}%")
                print(f"BLEU: {BLEU}, out of {total}: {100*BLEU/total}%")
        
        except Exception as e:
            print("title: ", title)
            print("Error in question number: ", total)
            print("Question: ", question)
            print("Error: ", e)
            errors.append(total)
            # total -= 1
            print("\n\n")
    
    if total != 0:
        print(f"Correct: {correct}, out of {total}: {100*correct/total}%")
        print(f"EM: {EM}, out of {total}: {100*EM/total}%")
        print(f"BLEU: {BLEU}, out of {total}: {100*BLEU/total}%")
    else:
        print("No Questions found with the given starting word")
    print("Errors: ", errors)
    return correct, total, errors, corrects


if __name__ == "__main__":
    initialize = Main()

    # dataset = load_dataset("rajpurkar/squad")
    # # dataset = load_dataset("quac")

    # # train and validation
    # train = dataset['train']
    # validation = dataset['validation']

    startsWith = ["who","what", "when", "where", "how many"]
    # b3dein what
    # x = QuestionStartsWith_Accuracy(initialize, train, startsWith)
    x = QuestionStartsWith_Accuracy(initialize, validation, startsWith)

    
    



In [ ]:
print("Accuracy: ", 100* x[0]/x[1])
print("Errors: ", x[2])
print("Corrects: ", x[3])
print("corrects: ", len(x[3]))
# corrects: [51, 79, 96, 137, 144, 159, 167, 181, 244, 260, 281, 297, 299, 300, 306, 307, 319, 322, 330, 362, 393, 434, 441, 450, 483, 507, 531, 537, 540, 564, 586, 587, 616, 653, 657, 683, 761, 765, 768, 795, 806, 842, 855, 876, 908, 919, 925, 948, 953, 961, 962, 984, 1001, 1003, 1004, 1082, 1086, 1103, 1110, 1132, 1176, 1258, 1329, 1345, 1389, 1392, 1421, 1424, 1427, 1442, 1446, 1448, 1470, 1477, 1482, 1501, 1511, 1528, 1534, 1567, 1613, 1650, 1673, 1692, 1694, 1695, 1698, 1708, 1709, 1717, 1721, 1764, 1772, 1806, 1820, 1834, 1836, 1885, 1936, 1958, 1959, 1965, 1975, 1985, 1986, 2042, 2068, 2076, 2105, 2108, 2146, 2150, 2258, 2279, 2303, 2315, 2328, 2348, 2355, 2357, 2360, 2370, 2375, 2389, 2404, 2407, 2429, 2465, 2470, 2484, 2499, 2528, 2556, 2559, 2590, 2644, 2645, 2650, 2653, 2655, 2658, 2693, 2695, 2699, 2732, 2733, 2734, 2739, 2757, 2792, 2802, 2803, 2804, 2810, 2813, 2831, 2926, 2938, 2942, 3028, 3031, 3040, 3066, 3086, 3120, 3183, 3189, 3200, 3202, 3208, 3214, 3227, 3257, 3299, 3341, 3350, 3368, 3421, 3446, 3466, 3477, 3514, 3524, 3529, 3534, 3563, 3570, 3581, 3591, 3594, 3610, 3615, 3617, 3623, 3645, 3652, 3716, 3718, 3747, 3758, 3762, 3784, 3795, 3800, 3801, 3803, 3804, 3807, 3819, 3831, 3835, 3852, 3860, 3862, 3880, 3944, 3955, 4105, 4117, 4119, 4126, 4134, 4136, 4143, 4169, 4174, 4239, 4291, 4320, 4328, 4379, 4408, 4477, 4530, 4537, 4557, 4560, 4573, 4580, 4587, 4599, 4605, 4618, 4622, 4632, 4638, 4654, 4655, 4656, 4660, 4663, 4705, 4759, 4760, 4770, 4779, 4781, 4788, 4799, 4803, 4804, 4809, 4811, 4819, 4825, 4829, 4843, 4854, 4862, 4867, 4872, 4886, 4893, 4921, 4947, 4953, 4955, 4957, 4961, 4965, 4977, 5017, 5020, 5041, 5042, 5044, 5047, 5055, 5057, 5086, 5090, 5098, 5107, 5109, 5122, 5137, 5138, 5139, 5141, 5157, 5183, 5191, 5213, 5216, 5218, 5303, 5314, 5315, 5325, 5341, 5350, 5352, 5356, 5365, 5367, 5373, 5375, 5376, 5398, 5405, 5406, 5414, 5436, 5439, 5447, 5456, 5484, 5504, 5505, 5578, 5664, 5670, 5683, 5686, 5699, 5705, 5752, 5777, 5785, 5817, 5821, 5843, 5849, 5851, 5870, 5875, 5916, 5920, 5937, 6012, 6016, 6017, 6018, 6029, 6055, 6059, 6065, 6082, 6098, 6152, 6155, 6167, 6177, 6178, 6180, 6184, 6186, 6188, 6190, 6194, 6195, 6199, 6203, 6205, 6206, 6207, 6215, 6217, 6218, 6219, 6235, 6259, 6281, 6287, 6306, 6311, 6313, 6320, 6332, 6343, 6347, 6348, 6349, 6350, 6375, 6377, 6383, 6385, 6415, 6422, 6434, 6447, 6449, 6450, 6453, 6478, 6486, 6493, 6503, 6507, 6510, 6512, 6515, 6516, 6526, 6527, 6558, 6570, 6605, 6628, 6632, 6657, 6693, 6702, 6707, 6732, 6739, 6741, 6774, 6806, 6816, 6838, 6841, 6845, 6848, 6851, 6860, 6871, 6879, 6900, 6904, 6911, 6980, 7018, 7030, 7034, 7043, 7047, 7084, 7085, 7088, 7091, 7108, 7133, 7139, 7144, 7167, 7170, 7183, 7186, 7188, 7192, 7211, 7217, 7222, 7235, 7247, 7253, 7274, 7275, 7280, 7294, 7311, 7329, 7368, 7387, 7413, 7415, 7426, 7429, 7433, 7440, 7452, 7461, 7462, 7464, 7465, 7467, 7469, 7472, 7474, 7475, 7478, 7483, 7487, 7488, 7491, 7494, 7497, 7499, 7501, 7502, 7505, 7508, 7511, 7519, 7520, 7522, 7534, 7539, 7552, 7554, 7562, 7564, 7571, 7578, 7587, 7589, 7597, 7600, 7603, 7715, 7733, 7734, 7766, 7790, 7802, 7805, 7812, 7851, 7858, 7870, 7906, 7908, 7942, 7948, 7981, 7984, 7985, 7986, 7987, 7988, 7993, 8016, 8019, 8022, 8024, 8029, 8039, 8041, 8058, 8065, 8088, 8093, 8098, 8102, 8140, 8145, 8149, 8177, 8188, 8219, 8255, 8266, 8268, 8280, 8281, 8282, 8284, 8295, 8311, 8338, 8400, 8403, 8509, 8523, 8524, 8525, 8535, 8541, 8553, 8558, 8579, 8618, 8649, 8693, 8696, 8713, 8717, 8721, 8724, 8734, 8745, 8747, 8796, 8797, 8803, 8805, 8810, 8875, 8888, 8925, 8960, 8982, 8985, 8987, 9003, 9004, 9010, 9056, 9093, 9100, 9102, 9104, 9111, 9114, 9119, 9123, 9125, 9128, 9130, 9134, 9136, 9137, 9139, 9150, 9153, 9158, 9159, 9167, 9171, 9178, 9186, 9191, 9193, 9196, 9228, 9279, 9283, 9293, 9324, 9356, 9359, 9371, 9372, 9378, 9379, 9391, 9452, 9453, 9457, 9458, 9460, 9463, 9469, 9510, 9511, 9523, 9524, 9539, 9575, 9576, 9577, 9581, 9591, 9595, 9616, 9617, 9618, 9620, 9621, 9622, 9654, 9663, 9665, 9667, 9668, 9669, 9679, 9682, 9683, 9709, 9736, 9756, 9761, 9763, 9767, 9833, 9838, 9844, 9847, 9852, 9862, 9878, 9894, 9906, 9911, 9920, 9921, 9926, 9945, 9946, 9948, 9955, 9956, 9974, 9982, 9985, 9997, 10001, 10004, 10033, 10055, 10056, 10071, 10088, 10098, 10123, 10139, 10140, 10162, 10185, 10186, 10189, 10216, 10226, 10255, 10302, 10326, 10340, 10341, 10342, 10343, 10354, 10373, 10385, 10420, 10477, 10547, 10568, 10593, 10594, 10610, 10613, 10614, 10618, 10624, 10653, 10654, 10662, 10664, 10671, 10675, 10776, 10788, 10800, 10801, 10807, 10810, 10857, 10858, 10881, 10892, 10899, 10901, 10910, 10916, 10944, 10967, 10971, 10972, 10984, 11041, 11086, 11090, 11091, 11105, 11116, 11139, 11172, 11191, 11193, 11220, 11236, 11258, 11262, 11273, 11290, 11297, 11308, 11309, 11313, 11316, 11324, 11340, 11349, 11362, 11368, 11369, 11377, 11413, 11490, 11552, 11619, 11623, 11663, 11678, 11747, 11762, 11783, 11788, 11816, 11849, 11869, 11891, 11897, 11900, 11902, 11966, 11980, 11995, 12032, 12052, 12078, 12094, 12095, 12122, 12131, 12135, 12141, 12148, 12156, 12206, 12209, 12212, 12236, 12281, 12312, 12317, 12325, 12341, 12357, 12358, 12394, 12410, 12411, 12418, 12425, 12473, 12514, 12525, 12530, 12550, 12570, 12596, 12597, 12603, 12625, 12631, 12645, 12651, 12652, 12719, 12729, 12738, 12746, 12750, 12754, 12779, 12780, 12797, 12799, 12804, 12818, 12848, 12867, 12893, 12902, 12913, 12915, 13075, 13079, 13080, 13095, 13118, 13130, 13143, 13144, 13145, 13152, 13155, 13172, 13174, 13189, 13193, 13206, 13299, 13308, 13322, 13330, 13331, 13337, 13348, 13350, 13363, 13365, 13392, 13394, 13427, 13441, 13474, 13495, 13522, 13547, 13549, 13598, 13672, 13696, 13733, 13778, 13805, 13806, 13822, 13836, 13849, 13856, 13859, 13861, 13954, 13955, 14026, 14052, 14054, 14058, 14086, 14122, 14132, 14133, 14144, 14185, 14187, 14192, 14193, 14200, 14205, 14212, 14213, 14215, 14225, 14233, 14295, 14296, 14307, 14317, 14330, 14349, 14350, 14361, 14365, 14383, 14395, 14396, 14417, 14421, 14422, 14427, 14439, 14440, 14445, 14447, 14452, 14453, 14456, 14474, 14488, 14504, 14511, 14532, 14555, 14619, 14628, 14634, 14655, 14698, 14724, 14749, 14753, 14856, 14862, 14865, 14874, 14914, 14924, 14935, 14967, 14980, 14993, 14999, 15010, 15013, 15017, 15019, 15040, 15052, 15098, 15105, 15113, 15151, 15154, 15156, 15193, 15212, 15252, 15259, 15266, 15275, 15295, 15301, 15303, 15306, 15314, 15316, 15319, 15322, 15327, 15349, 15382, 15388, 15447, 15476, 15484, 15501, 15518, 15519, 15524, 15537, 15548, 15551, 15571, 15588, 15598, 15619, 15650, 15666, 15679, 15697, 15725, 15727, 15755, 15757, 15769, 15771, 15782, 15809, 15822, 15823, 15834, 15858, 15866, 15879, 15880, 15882, 15885, 15886, 15887, 15894, 15895, 15896, 15925, 15934, 15949, 15952, 15973, 15996, 16031, 16053, 16090, 16093, 16101, 16103, 16105, 16133, 16150, 16172, 16212, 16214, 16217, 16218, 16245, 16249, 16288, 16305, 16307, 16314, 16354, 16381, 16437, 16440, 16442, 16562, 16593, 16599, 16604, 16647, 16653, 16657, 16661, 16690, 16728, 16736, 16744, 16766, 16817, 16847, 17061, 17072, 17075, 17076, 17090, 17174, 17234, 17236, 17262, 17269, 17272, 17371, 17376, 17377, 17433, 17451, 17507, 17530, 17564, 17705, 17731, 17732, 17771, 17782, 17783, 17796, 17799, 17806, 17807, 17809, 17822, 17835, 17850, 17855, 17863, 17877, 17886, 17915, 17930, 17937, 17957, 18027, 18052, 18056, 18073, 18090, 18092, 18093, 18095, 18097, 18107, 18110, 18115, 18116, 18118, 18128, 18130, 18133, 18137, 18140, 18141, 18149, 18153, 18168, 18173, 18205, 18234, 18353, 18374, 18383, 18398, 18411, 18450, 18454, 18455, 18461, 18484, 18496, 18500, 18583, 18590, 18639, 18701, 18754, 18755, 18807, 18818, 18823, 18830, 18832, 18920, 18921, 18960, 18999, 19039, 19042, 19044, 19082, 19083, 19126, 19151, 19164, 19165, 19169, 19179, 19228, 19238, 19244, 19264, 19273, 19294, 19325, 19328, 19329, 19335, 19396, 19442, 19445, 19463, 19494, 19501, 19511, 19542, 19617, 19620, 19631, 19635, 19656, 19685, 19724, 19744, 19782, 19878, 19898, 19924, 19927, 19939, 19943, 19951, 19954, 19967, 19968, 19971, 19987, 19989, 20003, 20025, 20026, 20041, 20044, 20069, 20143, 20145, 20146, 20165, 20183, 20185, 20190, 20209, 20214, 20224, 20230, 20249, 20286, 20330, 20336, 20402, 20418, 20428, 20454, 20472, 20513, 20516, 20521, 20530, 20531, 20547, 20567, 20660, 20665, 20685, 20702, 20713, 20730, 20751, 20777, 20791, 20792, 20862, 20922, 20932, 20957, 20985, 20988, 21014, 21023, 21026, 21032, 21037, 21038, 21112]
# dh kda mn 8eir what

# mn el vaidation: (msh bnbos root == root fl how many)
# Corrects:  [17, 48, 49, 59, 108, 111, 125, 132, 136, 150, 193, 218, 246, 251, 255, 258, 260, 268, 292, 326, 376, 395, 436, 442, 538, 564, 588, 599, 610, 611, 612, 618, 621, 625, 626, 627, 651, 665, 686, 696, 707, 714, 724, 734, 735, 766, 797, 799, 813, 824, 841, 927, 941, 994, 1021, 1052, 1077, 1092, 1096, 1126, 1135, 1272, 1286, 1328, 1349, 1437, 1644, 1701, 1729, 1730, 1774, 1800, 1851, 1855, 1861, 1870, 1896, 1926, 1956, 1976, 1979, 1988, 1993, 2103, 2146, 2148, 2153, 2160, 2169, 2170, 2191, 2204, 2207, 2220, 2287, 2288, 2296, 2309, 2469, 2497, 2500, 2517, 2569, 2627, 2756, 2814, 2822, 2855, 2897, 3003, 3013, 3047, 3049, 3067, 3082, 3083, 3090, 3106, 3212, 3280, 3288, 3329, 3346, 3451, 3491, 3503, 3504, 3539, 3543, 3603, 3607, 3615, 3629, 3641, 3665, 3692, 3714, 3715, 3739, 3767, 3790, 3807, 3812, 3839, 3842, 3850, 3918, 3927, 4027, 4038, 4040, 4044, 4048, 4057, 4063, 4067, 4091, 4105, 4145, 4150, 4167, 4192, 4240, 4266, 4284, 4327, 4402, 4605, 4638, 4646, 4667, 4687, 4777, 4794, 4804, 4811, 4812, 4813, 4825, 4884, 4897, 4952, 5020, 5043, 5060, 5067, 5099, 5109, 5138, 5156, 5188, 5199, 5202, 5218, 5222, 5257, 5259, 5296, 5323, 5376, 5400, 5503, 5528, 5532, 5559, 5560, 5567, 5656, 5729, 5798, 5807, 5815, 5836, 5839, 5853, 5867, 5873, 5877, 5905, 5907, 5912, 5913, 5974, 6029, 6154, 6175, 6243, 6250, 6298, 6340, 6522, 6525, 6548, 6594, 6600, 6602, 6607, 6608, 6648, 6683, 6700, 6711, 6729, 6746, 6752, 6760, 6769, 6800, 6809, 6814, 6826, 6841, 6864, 6868, 6884, 6894, 6910, 6974, 6977, 7008, 7023, 7026, 7052, 7086, 7102, 7123, 7133, 7163, 7166, 7168, 7171, 7175, 7176, 7211, 7227, 7249, 7250, 7272, 7278, 7313, 7330, 7425, 7452]
# Errors:  [3201]       # handled 5las

# dh b3d ma 7atet el condition bta3 root = root
# [17, 48, 108, 111, 125, 132, 136, 150, 218, 255, 292, 326, 376, 395, 436, 442, 538, 564, 588, 599, 610, 611, 618, 627, 651, 665, 686, 696, 707, 714, 724, 734, 735, 766, 797, 799, 813, 824, 841, 941, 994, 1021, 1052, 1077, 1092, 1096, 1126, 1135, 1272, 1328, 1349, 1437, 1644, 1701, 1729, 1730, 1774, 1800, 1851, 1855, 1861, 1870, 1896, 1926, 1956, 1976, 1979, 1988, 1993, 2103, 2146, 2148, 2153, 2160, 2169, 2191, 2204, 2207, 2220, 2287, 2288, 2296, 2309, 2469, 2497, 2500, 2517, 2569, 2627, 2756, 2814, 2822, 2855, 2897, 3003, 3013, 3047, 3049, 3067, 3082, 3083, 3090, 3106, 3201, 3212, 3280, 3288, 3329, 3346, 3451, 3491, 3503, 3504, 3539, 3543, 3603, 3607, 3615, 3629, 3641, 3665, 3692, 3714, 3715, 3739, 3767, 3790, 3807, 3812, 3839, 3850, 3918, 3927, 4027, 4038, 4040, 4044, 4048, 4057, 4063, 4067, 4091, 4105, 4145, 4150, 4167, 4192, 4240, 4266, 4284, 4327, 4402, 4605, 4638, 4646, 4667, 4687, 4777, 4794, 4804, 4811, 4812, 4813, 4825, 4884, 4897, 4952, 5020, 5043, 5060, 5067, 5099, 5109, 5138, 5156, 5188, 5199, 5202, 5218, 5222, 5257, 5259, 5296, 5323, 5376, 5400, 5528, 5567, 5656, 5729, 5798, 5807, 5815, 5836, 5839, 5853, 5867, 5873, 5877, 5905, 5907, 5912, 5913, 5974, 6029, 6154, 6243, 6250, 6298, 6340, 6522, 6525, 6548, 6594, 6600, 6602, 6607, 6608, 6648, 6683, 6700, 6711, 6729, 6746, 6752, 6760, 6769, 6800, 6809, 6814, 6826, 6841, 6864, 6868, 6884, 6894, 6910, 6974, 6977, 7008, 7023, 7026, 7052, 7086, 7102, 7123, 7133, 7163, 7166, 7168, 7171, 7175, 7176, 7211, 7227, 7249, 7250, 7272, 7278, 7313, 7330, 7425, 7452]

# dh lw shelt el condition menhom kolohom
# Corrects:  [17, 48, 49, 59, 87, 93, 108, 110, 111, 125, 129, 132, 150, 189, 193, 218, 234, 241, 245, 246, 251, 253, 255, 257, 258, 260, 267, 268, 270, 290, 292, 295, 298, 323, 326, 331, 334, 337, 339, 341, 369, 376, 385, 395, 436, 442, 479, 485, 503, 511, 517, 519, 521, 538, 543, 549, 559, 564, 588, 599, 610, 611, 612, 618, 621, 625, 626, 627, 643, 651, 665, 682, 686, 696, 707, 709, 714, 724, 729, 734, 735, 766, 795, 796, 797, 799, 808, 813, 815, 824, 841, 846, 866, 867, 871, 886, 892, 903, 905, 906, 915, 917, 923, 925, 927, 934, 935, 941, 946, 953, 956, 963, 964, 979, 985, 986, 987, 988, 994, 996, 999, 1001, 1003, 1006, 1008, 1010, 1016, 1019, 1021, 1024, 1039, 1052, 1063, 1077, 1096, 1099, 1111, 1114, 1126, 1132, 1135, 1137, 1142, 1148, 1152, 1154, 1164, 1165, 1166, 1171, 1176, 1178, 1188, 1192, 1198, 1199, 1201, 1206, 1209, 1216, 1230, 1237, 1239, 1249, 1272, 1279, 1280, 1286, 1287, 1291, 1299, 1304, 1308, 1313, 1314, 1315, 1328, 1349, 1354, 1372, 1414, 1437, 1450, 1457, 1468, 1477, 1494, 1496, 1521, 1545, 1546, 1552, 1569, 1604, 1609, 1617, 1621, 1625, 1628, 1637, 1642, 1644, 1672, 1676, 1678, 1683, 1690, 1695, 1696, 1698, 1700, 1701, 1702, 1704, 1705, 1708, 1714, 1728, 1729, 1730, 1749, 1751, 1763, 1765, 1768, 1774, 1777, 1778, 1782, 1786, 1789, 1792, 1797, 1799, 1800, 1816, 1818, 1830, 1831, 1841, 1846, 1851, 1855, 1860, 1861, 1870, 1871, 1887, 1891, 1893, 1896, 1905, 1917, 1924, 1926, 1929, 1932, 1937, 1938, 1939, 1947, 1948, 1952, 1956, 1961, 1972, 1976, 1979, 1982, 1983, 1990, 1992, 1993, 1999, 2003, 2006, 2030, 2042, 2088, 2089, 2103, 2132, 2134, 2145, 2146, 2148, 2153, 2156, 2160, 2169, 2170, 2175, 2181, 2191, 2197, 2200, 2201, 2204, 2207, 2211, 2220, 2252, 2254, 2260, 2268, 2272, 2274, 2287, 2288, 2292, 2296, 2309, 2313, 2314, 2341, 2378, 2419, 2431, 2451, 2452, 2469, 2490, 2497, 2500, 2510, 2517, 2526, 2552, 2553, 2560, 2569, 2609, 2610, 2612, 2613, 2623, 2627, 2631, 2639, 2655, 2663, 2665, 2678, 2679, 2683, 2685, 2710, 2712, 2725, 2734, 2756, 2762, 2765, 2781, 2788, 2789, 2797, 2798, 2814, 2822, 2826, 2843, 2851, 2855, 2872, 2891, 2893, 2897, 2910, 2960, 2974, 3003, 3013, 3014, 3018, 3041, 3043, 3044, 3047, 3049, 3063, 3064, 3067, 3068, 3080, 3082, 3083, 3088, 3090, 3095, 3106, 3201, 3209, 3212, 3216, 3247, 3252, 3257, 3263, 3280, 3284, 3288, 3292, 3315, 3318, 3325, 3329, 3333, 3346, 3348, 3374, 3396, 3397, 3465, 3475, 3480, 3482, 3487, 3491, 3503, 3504, 3524, 3543, 3603, 3607, 3615, 3625, 3629, 3649, 3665, 3666, 3684, 3692, 3696, 3702, 3715, 3739, 3751, 3767, 3790, 3797, 3799, 3801, 3807, 3812, 3826, 3835, 3839, 3842, 3850, 3918, 3921, 3923, 3927, 3958, 3960, 3979, 3989, 3993, 3997, 3999, 4006, 4013, 4023, 4027, 4029, 4036, 4038, 4040, 4044, 4048, 4049, 4057, 4063, 4067, 4069, 4080, 4084, 4091, 4092, 4096, 4097, 4098, 4105, 4107, 4111, 4117, 4126, 4130, 4144, 4145, 4150, 4155, 4160, 4167, 4181, 4184, 4186, 4192, 4202, 4204, 4205, 4240, 4246, 4253, 4266, 4271, 4282, 4284, 4295, 4297, 4314, 4319, 4327, 4340, 4367, 4398, 4402, 4403, 4406, 4419, 4423, 4425, 4427, 4431, 4447, 4450, 4457, 4528, 4533, 4541, 4544, 4557, 4573, 4575, 4590, 4592, 4598, 4605, 4638, 4646, 4651, 4658, 4667, 4687, 4707, 4750, 4787, 4794, 4799, 4801, 4812, 4813, 4825, 4830, 4884, 4897, 4929, 4941, 4951, 4952, 4953, 4984, 4987, 5020, 5031, 5043, 5060, 5063, 5067, 5080, 5099, 5100, 5107, 5109, 5138, 5147, 5156, 5160, 5175, 5188, 5189, 5199, 5200, 5202, 5208, 5218, 5222, 5224, 5229, 5234, 5239, 5257, 5259, 5262, 5279, 5281, 5296, 5308, 5323, 5330, 5346, 5349, 5350, 5377, 5382, 5393, 5400, 5408, 5414, 5427, 5448, 5462, 5470, 5474, 5497, 5499, 5502, 5503, 5518, 5526, 5527, 5528, 5532, 5534, 5559, 5560, 5562, 5567, 5568, 5569, 5570, 5594, 5595, 5640, 5642, 5646, 5647, 5656, 5660, 5667, 5669, 5677, 5702, 5707, 5709, 5722, 5729, 5755, 5761, 5763, 5769, 5775, 5780, 5784, 5788, 5798, 5800, 5802, 5806, 5807, 5809, 5815, 5819, 5834, 5836, 5839, 5840, 5851, 5853, 5867, 5870, 5873, 5877, 5878, 5886, 5894, 5900, 5903, 5905, 5907, 5912, 5913, 5921, 5947, 5970, 5974, 5984, 6016, 6029, 6033, 6057, 6059, 6060, 6061, 6085, 6086, 6088, 6089, 6093, 6100, 6105, 6125, 6135, 6138, 6140, 6146, 6154, 6160, 6161, 6168, 6175, 6188, 6191, 6227, 6243, 6244, 6250, 6251, 6252, 6253, 6271, 6273, 6279, 6281, 6286, 6298, 6311, 6314, 6356, 6358, 6382, 6390, 6391, 6419, 6464, 6483, 6488, 6489, 6496, 6515, 6522, 6525, 6537, 6544, 6548, 6552, 6553, 6588, 6594, 6599, 6600, 6602, 6607, 6608, 6619, 6648, 6649, 6660, 6661, 6673, 6678, 6683, 6688, 6699, 6700, 6707, 6729, 6746, 6752, 6760, 6769, 6799, 6800, 6808, 6809, 6814, 6817, 6821, 6823, 6826, 6835, 6841, 6847, 6862, 6864, 6865, 6868, 6877, 6880, 6884, 6894, 6903, 6907, 6910, 6919, 6920, 6924, 6930, 6932, 6938, 6953, 6959, 6963, 6964, 6967, 6974, 6976, 6977, 6983, 7004, 7007, 7008, 7011, 7023, 7026, 7027, 7029, 7034, 7049, 7052, 7056, 7060, 7072, 7086, 7090, 7096, 7100, 7101, 7102, 7123, 7133, 7163, 7166, 7168, 7171, 7175, 7176, 7187, 7194, 7209, 7211, 7213, 7226, 7227, 7233, 7238, 7246, 7249, 7272, 7278, 7311, 7313, 7324, 7330, 7335, 7347, 7349, 7366, 7382, 7401, 7404, 7416, 7417, 7421, 7425, 7431, 7439, 7452, 7462, 7479, 7480]
# Accuracy:  11.640635429181685
# corrects:  872

# 2a5er 7aga, b 7awr eno y3ml loosen lel constraints
# Accuracy:  11.424017003188098
# Errors:  []
# Corrects:  [17, 48, 49, 59, 87, 93, 108, 110, 111, 125, 129, 132, 136, 150, 189, 193, 218, 234, 241, 245, 246, 251, 253, 255, 257, 258, 260, 267, 268, 270, 290, 292, 295, 298, 323, 326, 331, 334, 337, 339, 341, 369, 376, 385, 395, 437, 443, 480, 486, 504, 512, 518, 520, 522, 539, 545, 552, 562, 567, 591, 602, 613, 614, 615, 621, 624, 628, 629, 630, 646, 654, 668, 685, 689, 699, 710, 712, 717, 727, 732, 738, 739, 770, 799, 800, 801, 803, 812, 817, 819, 828, 846, 872, 873, 877, 892, 898, 909, 911, 912, 921, 923, 929, 931, 933, 940, 941, 947, 952, 959, 962, 969, 970, 985, 992, 993, 994, 1000, 1002, 1005, 1008, 1010, 1013, 1015, 1017, 1023, 1026, 1028, 1031, 1046, 1059, 1070, 1084, 1099, 1103, 1106, 1118, 1121, 1133, 1139, 1142, 1144, 1149, 1155, 1159, 1161, 1171, 1172, 1173, 1178, 1183, 1195, 1199, 1205, 1206, 1208, 1213, 1216, 1223, 1237, 1244, 1246, 1256, 1279, 1286, 1287, 1293, 1294, 1298, 1306, 1311, 1315, 1321, 1322, 1336, 1357, 1362, 1381, 1423, 1446, 1459, 1466, 1477, 1486, 1503, 1505, 1530, 1554, 1555, 1561, 1578, 1618, 1626, 1630, 1634, 1637, 1646, 1651, 1653, 1681, 1685, 1688, 1693, 1700, 1705, 1706, 1708, 1710, 1711, 1712, 1714, 1715, 1718, 1724, 1725, 1740, 1741, 1742, 1761, 1763, 1775, 1777, 1780, 1786, 1789, 1790, 1794, 1798, 1801, 1804, 1809, 1811, 1812, 1828, 1830, 1842, 1843, 1853, 1858, 1863, 1867, 1872, 1873, 1882, 1883, 1900, 1904, 1906, 1909, 1918, 1919, 1931, 1938, 1940, 1943, 1946, 1951, 1952, 1953, 1961, 1962, 1970, 1975, 1986, 1990, 1993, 1997, 2002, 2004, 2006, 2007, 2014, 2018, 2021, 2045, 2057, 2103, 2118, 2147, 2149, 2160, 2161, 2163, 2168, 2171, 2175, 2185, 2186, 2191, 2197, 2207, 2213, 2216, 2217, 2220, 2223, 2227, 2236, 2270, 2276, 2284, 2288, 2290, 2303, 2304, 2308, 2312, 2325, 2329, 2330, 2357, 2394, 2435, 2447, 2467, 2468, 2485, 2506, 2513, 2516, 2526, 2533, 2542, 2568, 2569, 2576, 2585, 2625, 2626, 2628, 2629, 2639, 2643, 2647, 2655, 2671, 2679, 2681, 2694, 2695, 2699, 2726, 2728, 2741, 2750, 2772, 2778, 2781, 2797, 2804, 2805, 2813, 2814, 2830, 2838, 2842, 2859, 2867, 2871, 2888, 2907, 2909, 2913, 2926, 2976, 2990, 3019, 3029, 3030, 3034, 3057, 3059, 3060, 3063, 3065, 3079, 3080, 3083, 3084, 3096, 3098, 3099, 3104, 3106, 3111, 3122, 3217, 3225, 3228, 3232, 3268, 3279, 3296, 3300, 3304, 3308, 3331, 3334, 3341, 3345, 3349, 3362, 3364, 3390, 3412, 3413, 3467, 3491, 3496, 3498, 3503, 3507, 3519, 3520, 3540, 3555, 3559, 3589, 3621, 3625, 3633, 3644, 3648, 3660, 3668, 3684, 3685, 3711, 3715, 3721, 3733, 3734, 3758, 3770, 3786, 3810, 3817, 3819, 3821, 3827, 3832, 3846, 3855, 3859, 3862, 3870, 3938, 3941, 3943, 3948, 3983, 3985, 4004, 4014, 4018, 4022, 4024, 4031, 4038, 4048, 4052, 4054, 4061, 4063, 4065, 4069, 4073, 4074, 4082, 4088, 4092, 4094, 4105, 4109, 4116, 4117, 4121, 4122, 4123, 4130, 4132, 4136, 4142, 4151, 4155, 4169, 4170, 4175, 4180, 4185, 4192, 4206, 4209, 4211, 4217, 4227, 4229, 4230, 4265, 4271, 4278, 4291, 4307, 4309, 4320, 4322, 4339, 4344, 4353, 4366, 4393, 4425, 4429, 4430, 4433, 4446, 4450, 4454, 4458, 4474, 4477, 4484, 4556, 4561, 4572, 4585, 4601, 4603, 4618, 4620, 4626, 4633, 4666, 4674, 4679, 4686, 4695, 4715, 4735, 4778, 4805, 4815, 4822, 4827, 4829, 4832, 4840, 4841, 4842, 4854, 4859, 4913, 4926, 4970, 4980, 4981, 4982, 5013, 5016, 5049, 5060, 5072, 5089, 5092, 5096, 5109, 5128, 5129, 5136, 5138, 5167, 5185, 5189, 5204, 5217, 5218, 5228, 5229, 5231, 5237, 5247, 5251, 5253, 5258, 5263, 5268, 5286, 5288, 5291, 5308, 5310, 5325, 5337, 5352, 5359, 5375, 5378, 5379, 5405, 5406, 5411, 5422, 5429, 5437, 5443, 5456, 5477, 5491, 5499, 5503, 5526, 5528, 5531, 5532, 5547, 5555, 5556, 5557, 5561, 5563, 5588, 5589, 5591, 5596, 5597, 5598, 5599, 5623, 5624, 5669, 5671, 5675, 5676, 5685, 5689, 5696, 5698, 5706, 5731, 5736, 5738, 5751, 5758, 5784, 5790, 5792, 5798, 5804, 5809, 5813, 5817, 5827, 5829, 5831, 5835, 5836, 5838, 5844, 5848, 5863, 5865, 5868, 5869, 5880, 5882, 5896, 5899, 5902, 5906, 5907, 5915, 5923, 5929, 5932, 5934, 5936, 5941, 5942, 5950, 5976, 5999, 6003, 6013, 6045, 6058, 6062, 6086, 6088, 6089, 6090, 6114, 6115, 6117, 6118, 6129, 6135, 6155, 6176, 6184, 6190, 6191, 6198, 6205, 6218, 6221, 6257, 6273, 6274, 6280, 6281, 6282, 6283, 6301, 6309, 6311, 6316, 6328, 6341, 6344, 6370, 6386, 6388, 6420, 6421, 6449, 6494, 6513, 6518, 6519, 6526, 6545, 6552, 6555, 6567, 6574, 6578, 6582, 6618, 6624, 6629, 6630, 6632, 6637, 6638, 6649, 6678, 6679, 6690, 6691, 6708, 6713, 6719, 6730, 6731, 6738, 6742, 6760, 6777, 6783, 6791, 6800, 6830, 6831, 6839, 6840, 6845, 6848, 6852, 6854, 6857, 6866, 6872, 6878, 6893, 6895, 6896, 6899, 6908, 6911, 6915, 6925, 6936, 6941, 6944, 6953, 6954, 6958, 6964, 6966, 6972, 6987, 6993, 6997, 6998, 7001, 7008, 7010, 7011, 7017, 7038, 7041, 7042, 7045, 7057, 7060, 7061, 7063, 7068, 7083, 7086, 7090, 7094, 7106, 7120, 7124, 7130, 7134, 7135, 7136, 7157, 7167, 7197, 7200, 7202, 7205, 7209, 7210, 7221, 7228, 7243, 7245, 7247, 7260, 7261, 7267, 7272, 7280, 7283, 7284, 7307, 7313, 7346, 7348, 7359, 7365, 7370, 7384, 7386, 7403, 7419, 7441, 7453, 7454, 7458, 7462, 7468, 7489, 7499, 7516, 7517]
# corrects:  860

# dh howa howa bs b kolo badal total
# Accuracy:  11.424017003188098
# Errors:  []
# Corrects:  [22, 61, 62, 72, 110, 117, 136, 138, 139, 158, 167, 172, 176, 194, 241, 245, 275, 292, 301, 305, 306, 311, 313, 315, 317, 318, 320, 329, 330, 332, 355, 358, 364, 369, 402, 405, 410, 413, 421, 423, 425, 464, 472, 492, 502, 558, 565, 622, 629, 651, 661, 669, 671, 673, 694, 704, 713, 724, 731, 760, 772, 786, 787, 788, 795, 798, 802, 803, 804, 826, 834, 851, 872, 876, 888, 900, 902, 907, 919, 925, 933, 934, 972, 1004, 1006, 1007, 1009, 1019, 1026, 1028, 1038, 1059, 1087, 1088, 1092, 1108, 1114, 1126, 1128, 1129, 1139, 1141, 1147, 1149, 1151, 1158, 1159, 1166, 1173, 1184, 1187, 1196, 1197, 1215, 1222, 1223, 1225, 1234, 1236, 1240, 1243, 1246, 1249, 1252, 1255, 1264, 1267, 1270, 1275, 1295, 1312, 1323, 1338, 1358, 1364, 1367, 1381, 1384, 1397, 1408, 1412, 1414, 1422, 1428, 1432, 1434, 1447, 1448, 1449, 1456, 1461, 1478, 1483, 1490, 1491, 1493, 1498, 1507, 1515, 1533, 1543, 1547, 1558, 1587, 1601, 1602, 1614, 1615, 1620, 1628, 1634, 1639, 1647, 1648, 1667, 1692, 1700, 1734, 1818, 1849, 1874, 1881, 1893, 1904, 1924, 1926, 1958, 1985, 1987, 1995, 2020, 2066, 2074, 2079, 2085, 2088, 2102, 2108, 2110, 2147, 2152, 2156, 2161, 2175, 2183, 2184, 2186, 2188, 2189, 2190, 2192, 2193, 2196, 2204, 2206, 2223, 2224, 2225, 2252, 2255, 2267, 2269, 2273, 2282, 2287, 2288, 2293, 2298, 2301, 2307, 2313, 2315, 2316, 2336, 2338, 2355, 2356, 2369, 2376, 2381, 2386, 2391, 2392, 2406, 2407, 2431, 2436, 2438, 2441, 2451, 2452, 2465, 2472, 2474, 2477, 2480, 2485, 2486, 2487, 2496, 2498, 2510, 2516, 2529, 2534, 2537, 2541, 2548, 2550, 2552, 2553, 2561, 2566, 2570, 2600, 2620, 2719, 2751, 2805, 2807, 2819, 2820, 2822, 2827, 2830, 2834, 2845, 2846, 2851, 2858, 2868, 2874, 2877, 2878, 2881, 2888, 2893, 2903, 2944, 2953, 2963, 2968, 2970, 2988, 2989, 2994, 3001, 3017, 3022, 3023, 3062, 3111, 3172, 3187, 3217, 3219, 3243, 3275, 3287, 3291, 3311, 3321, 3334, 3368, 3369, 3378, 3389, 3459, 3460, 3463, 3464, 3481, 3492, 3497, 3515, 3545, 3555, 3559, 3577, 3578, 3582, 3642, 3647, 3665, 3677, 3716, 3725, 3731, 3776, 3791, 3792, 3818, 3822, 3840, 3849, 3854, 3879, 3892, 3896, 3917, 3953, 3955, 3959, 3974, 4045, 4070, 4113, 4129, 4132, 4138, 4185, 4188, 4189, 4199, 4201, 4228, 4229, 4234, 4235, 4251, 4256, 4259, 4277, 4282, 4297, 4317, 4477, 4490, 4493, 4498, 4544, 4572, 4595, 4599, 4604, 4608, 4648, 4654, 4671, 4680, 4686, 4711, 4713, 4760, 4792, 4793, 4878, 4917, 4924, 4926, 4933, 4938, 4951, 4952, 4975, 5001, 5005, 5056, 5116, 5124, 5141, 5152, 5158, 5172, 5181, 5198, 5199, 5237, 5242, 5251, 5263, 5264, 5291, 5304, 5323, 5349, 5356, 5358, 5360, 5366, 5372, 5391, 5400, 5404, 5412, 5426, 5572, 5580, 5582, 5601, 5702, 5704, 5731, 5744, 5751, 5756, 5758, 5767, 5784, 5795, 5799, 5801, 5812, 5816, 5819, 5823, 5828, 5829, 5840, 5846, 5852, 5855, 5876, 5882, 5891, 5892, 5896, 5897, 5899, 5908, 5911, 5916, 5931, 5943, 5950, 5969, 5971, 5976, 5981, 5988, 5996, 6017, 6021, 6025, 6033, 6049, 6053, 6059, 6110, 6118, 6127, 6145, 6166, 6169, 6185, 6188, 6211, 6216, 6232, 6255, 6294, 6349, 6354, 6355, 6358, 6375, 6381, 6386, 6390, 6409, 6412, 6419, 6510, 6516, 6530, 6552, 6579, 6583, 6611, 6613, 6629, 6646, 6692, 6700, 6707, 6724, 6741, 6780, 6812, 6879, 6918, 6938, 6961, 6967, 6970, 6974, 6989, 6990, 6991, 7006, 7014, 7102, 7124, 7194, 7207, 7211, 7212, 7255, 7263, 7311, 7327, 7346, 7365, 7369, 7376, 7392, 7413, 7414, 7422, 7424, 7459, 7479, 7486, 7502, 7523, 7525, 7539, 7540, 7542, 7548, 7560, 7565, 7568, 7574, 7580, 7587, 7611, 7614, 7619, 7640, 7644, 7665, 7679, 7697, 7706, 7729, 7732, 7733, 7779, 7780, 7785, 7797, 7809, 7821, 7828, 7850, 7876, 7896, 7908, 7914, 7944, 7946, 7949, 7950, 7969, 7987, 7988, 7989, 7994, 7999, 8034, 8035, 8037, 8042, 8045, 8046, 8052, 8084, 8085, 8131, 8133, 8137, 8139, 8148, 8153, 8160, 8162, 8170, 8197, 8202, 8204, 8218, 8228, 8260, 8266, 8272, 8282, 8288, 8293, 8297, 8304, 8315, 8317, 8319, 8324, 8325, 8327, 8333, 8343, 8360, 8363, 8366, 8367, 8381, 8384, 8401, 8404, 8407, 8411, 8412, 8421, 8429, 8437, 8440, 8442, 8444, 8451, 8452, 8461, 8495, 8521, 8526, 8536, 8575, 8588, 8592, 8623, 8625, 8627, 8628, 8654, 8655, 8657, 8658, 8674, 8680, 8705, 8730, 8738, 8747, 8748, 8755, 8763, 8776, 8779, 8826, 8846, 8847, 8855, 8856, 8857, 8860, 8880, 8889, 8891, 8896, 8909, 8923, 8927, 8957, 8975, 8977, 9020, 9021, 9067, 9141, 9165, 9173, 9174, 9183, 9211, 9222, 9227, 9242, 9253, 9258, 9262, 9315, 9321, 9326, 9327, 9332, 9338, 9341, 9358, 9396, 9397, 9409, 9410, 9434, 9440, 9448, 9460, 9461, 9469, 9473, 9496, 9518, 9525, 9534, 9551, 9586, 9587, 9601, 9603, 9609, 9612, 9617, 9620, 9623, 9635, 9641, 9648, 9669, 9671, 9673, 9676, 9689, 9693, 9699, 9713, 9726, 9731, 9734, 9747, 9749, 9753, 9760, 9764, 9772, 9790, 9797, 9803, 9804, 9807, 9830, 9836, 9841, 9859, 9894, 9897, 9898, 9903, 9922, 9926, 9927, 9929, 9938, 9961, 9966, 9970, 9974, 9993, 10011, 10019, 10025, 10031, 10033, 10035, 10060, 10077, 10141, 10150, 10152, 10156, 10161, 10162, 10192, 10200, 10220, 10222, 10224, 10242, 10246, 10253, 10259, 10267, 10271, 10272, 10303, 10312, 10354, 10357, 10369, 10375, 10380, 10394, 10396, 10421, 10441, 10468, 10483, 10484, 10488, 10492, 10499, 10523, 10535, 10555, 10556]
# corrects:  860

In [ ]:
# from datasets import load_dataset

# dataset = load_dataset("rajpurkar/squad")
# # dataset = load_dataset("quac")

# # train and validation
# train = dataset['train']
# validation = dataset['validation']

# # save them to csv
# # train.to_csv('dataset/train.csv')
# # validation.to_csv('dataset/validation.csv')


In [2]:
import getopt
import sys

from kwQnA._exportPairs import exportToJSON
from kwQnA._getentitypair import GetEntity
from kwQnA._graph import GraphEnt
from kwQnA._qna import QuestionAnswer

import pandas as pd
from datasets import load_dataset
import json 
import regex as re
from tqdm import tqdm
import nltk


In [3]:

class Main:
    """docstring for Main."""

    def __init__(self):
        super(Main, self).__init__()
        self.qna = QuestionAnswer()
        self.getEntity = GetEntity()
        self.export = exportToJSON()
        self.graph = GraphEnt()

    def main(self, argv):
        inputfile = ''
        inputQue = ''
        try:
            opts, args = getopt.getopt(argv, "hi:q:g:s:", ["ifile=", "question=","showGraph=","showEntities="])
            if opts == [] and args == []:
                print("ERROR")
                print("Help:")
                print("python init.py -i <TextFileName> -q <Question> -s <show Ent>")
        except getopt.GetoptError as err:
            sys.exit(2)

        for opt, arg in opts:
            showGraph , showEntities= "f", "f"
            if opt == '-h':
                print ('test.py -i <inputfile> -q <question> -g <y or n> -s <Show Entities>')
                sys.exit()
            elif opt in ("-i", "--ifile"):
                inputfile = arg
            elif opt in ("-q", "--question"):
                inputQue = arg
            elif opt in ("-g", "--showGraph"):
                showGraph = arg
            elif opt in ("-s", "--showEntities"):
                showEntities = arg
            else:
                assert False, "unhandled option"

        return inputfile, inputQue, showGraph, showEntities


def QuestionStartsWith_Accuracy(initialize, dataset, startsWith):

    correct = 0
    EM = 0
    BLEU = 0
    total = 0
    errors = []
    corrects = []
    kolo = 0
    for item in tqdm(dataset):
        random_number = random.randint(0, len(dataset))
        kolo += 1
        included_question = False
        try:
            context = item['context'].lower()
            context = re.sub(' +', ' ', context)
            
            question = item['question'].lower()
            question = re.sub(' +', ' ', question)
            
            answer = item['answers']['text'][0].lower()
            title = item['title'].lower()
    
            # check if question starts with startsWith
            for start in startsWith:
                if question.startswith(start.lower()):
                    included_question = True
                    break
            
            if included_question:
    
                total += 1
                refined_text = initialize.getEntity.preprocess_context(context, title)
                dataEntities, numberOfPairs = initialize.getEntity.get_entity(refined_text)
                if dataEntities:
                    initialize.export.dumpdata(dataEntities[0])

                    
                    if answer == 'antiforms':
                        print("the question isssssssssssssssssssss: ", question)
                        
                    outputAnswer = initialize.qna.findanswer(question)
                    if list(outputAnswer)[0] == 'None':
                        continue
                    
                    found = False
                    currMaxBleu = 0
                    for ans in outputAnswer:
                        if found:
                            break
                        
                        if ans in answer or answer in ans:
                            correct += 1
                            # print("answer: ", answer , "ans: " , ans , "outputAnswer: ", outputAnswer)
                            corrects.append(kolo)
                            found = True
                        if ans == answer:
                            EM += 1
                            found = True

                        n = min(len(ans.split()), 4)
                        if n == 0:
                            BLEUscore = 0
                        else:
                            weights = [1.0/n]*n
                            BLEUscore = nltk.translate.bleu_score.sentence_bleu([answer], ans, weights=weights)
                            currMaxBleu = max(BLEUscore, currMaxBleu)
                        # if BLEUscore > 0.5 and ans != 'None':
                        #     BLEU += 1
                        #     found = True
                    BLEU += currMaxBleu
                        

            if kolo % 1000 == 0:
                print(f"Correct: {correct}, out of {total}: {100*correct/total}%")
                print(f"EM: {EM}, out of {total}: {100*EM/total}%")
                print(f"BLEU: {BLEU}, out of {total}: {100*BLEU/total}%")
        
        except Exception as e:
            print("title: ", title)
            print("Error in question number: ", total)
            print("Question: ", question)
            print("Error: ", e)
            errors.append(total)
            # total -= 1
            print("\n\n")
    
    if total != 0:
        print(f"Correct: {correct}, out of {total}: {100*correct/total}%")
        print(f"EM: {EM}, out of {total}: {100*EM/total}%")
        print(f"BLEU: {BLEU}, out of {total}: {100*BLEU/total}%")
    else:
        print("No Questions found with the given starting word")
    print("Errors: ", errors)
    return correct, total, errors, corrects


if __name__ == "__main__":
    initialize = Main()

    # dataset = load_dataset("rajpurkar/squad")
    # # dataset = load_dataset("quac")

    # # train and validation
    # train = dataset['train']
    # validation = dataset['validation']

    startsWith = ["what"]
    # b3dein what
    # x = QuestionStartsWith_Accuracy(initialize, train, startsWith)
    x = QuestionStartsWith_Accuracy(initialize, validation, startsWith)



  0%|          | 5/10570 [00:00<04:55, 35.74it/s]c:\Users\zisak\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\zisak\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
  0%|          | 9/10570 [00:00<12:14, 14.38it/s]c:\Users\zisak\anaconda3\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps

Correct: 94, out of 851: 11.045828437132785%
EM: 22, out of 851: 2.5851938895417157%
BLEU: 76.55643394676775, out of 851: 8.99605569292218%


 28%|██▊       | 3003/10570 [03:04<04:57, 25.41it/s]

Correct: 174, out of 1340: 12.985074626865671%
EM: 38, out of 1340: 2.8358208955223883%
BLEU: 141.9051634557372, out of 1340: 10.589937571323672%


 38%|███▊      | 4002/10570 [03:59<05:37, 19.47it/s]

...

Correct: 324, out of 2558: 12.666145426114152%
EM: 66, out of 2558: 2.580140734949179%
BLEU: 243.70382838347663, out of 2558: 9.527123861746546%


 66%|██████▌   | 7000/10570 [07:29<02:23, 24.81it/s]

Correct: 388, out of 3044: 12.746386333771353%
EM: 80, out of 3044: 2.628120893561104%
BLEU: 287.2957152819773, out of 3044: 9.438098399539333%


100%|██████████| 10570/10570 [11:55<00:00, 14.77it/s]

Correct: 625, out of 4763: 13.121981944152845%
EM: 120, out of 4763: 2.519420533277346%
BLEU: 444.1112224301423, out of 4763: 9.32419110707836%
Errors:  []


In [9]:
nltk.translate.bleu_score.sentence_bleu(['seven'],'seven times' , weights=[0.25,0.25])

0.6529942057256104